In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


import torchvision.transforms as transforms
import torchvision

from PIL import Image

random_seed = 42
BATCH_SIZE = 4

# Get data (chexpert)

with help from https://www.kaggle.com/hmchuong/chexpert-pytorch-densenet121/?select=test.csv

In [4]:
path_folder = "/Users/carlasagebiel/Documents/AML_project/data/"
weight_path = "/Users/carlasagebiel/Documents/AML_project/_weights"

In [5]:
train_path = "/Users/carlasagebiel/Documents/AML_project/data/CheXpert-v1.0-small/train"
valid_path = "/Users/carlasagebiel/Documents/AML_project/data/CheXpert-v1.0-small/valid"

In [6]:
train_data = pd.read_csv("/Users/carlasagebiel/Documents/AML_project/data/CheXpert-v1.0-small/train.csv")
valid_data = pd.read_csv("/Users/carlasagebiel/Documents/AML_project/data/CheXpert-v1.0-small/valid.csv")

In [7]:
LABELS = train_data.columns[5:]
LABELS 

Index(['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices'],
      dtype='object')

In [8]:
train, test = train_test_split(train_data, test_size= 0.1, random_state = random_seed)

In [9]:
class ChestXrayDataset(Dataset):
    
    def __init__(self, folder_dir, dataframe, image_size = 256):
        """
        Init Dataset
        
        Parameters
        ----------
        folder_dir: str
            folder contains all images
        dataframe: pandas.DataFrame
            dataframe contains all information of images
        image_size: int
            image size to rescale
        normalization: bool
            whether applying normalization with mean and std from ImageNet or not
        """
        self.image_paths = [] # List of image paths
        self.image_labels = [] # List of image labels
        
        # Define list of image transformations
        image_transformation = [
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ]
        
        
        self.image_transformation = transforms.Compose(image_transformation)
        
        # Get all image paths and image labels from dataframe
        for index, row in dataframe.iterrows():
            image_path = os.path.join(folder_dir, row.Path)
            self.image_paths.append(image_path)
            if len(row) < 14:
                labels = [0] * 14
            else:
                labels = []
                for col in row[5:]:
                    if col == 1:
                        labels.append(1)
                    else:
                        labels.append(0)
            self.image_labels.append(labels)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        """
        Read image at index and convert to torch Tensor
        """
        
        # Read image
        image_path = self.image_paths[index]
        image_data = Image.open(image_path)#.convert("RGB") # Convert image to RGB channels
        
        # Resize and convert image to torch tensor 
        image_data = self.image_transformation(image_data)
        
        return image_data, torch.FloatTensor(self.image_labels[index])

In [10]:
train_dataset = ChestXrayDataset(path_folder, train_data)
val_dataset = ChestXrayDataset(path_folder, valid_data)

In [12]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle = True)

# Get model

In [13]:
from network.feature_extractor import ResNet
from network.training import PretrainTraining

In [14]:
model1111 = ResNet(dims = [1,1,1,1], out_shape= 14)
model2222 = ResNet(dims = [2,2,2,2], out_shape= 14)
model1283 = ResNet(dims = [1,2,8,3], out_shape= 14)
model3463 = ResNet(dims = [3,4,6,3], out_shape= 14)

In [15]:
loss = nn.CrossEntropyLoss()

In [16]:
training = PretrainTraining("resnet1111", model1111, loss,
                            batch_size= BATCH_SIZE, verbose_level= 2, path_dir = weight_path,
                            data_trafo= False)

/Users/carlasagebiel/Documents/AML_project/network/training.py:47: UserWarning: Data trafos are not specified. It can't be stored in net_config.json
  warn("Data trafos are not specified."


In [17]:
training.train(3, train_dataloader)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/55854 [00:00<?, ?it/s]

/Users/carlasagebiel/Documents/AML_project/.venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 